In [1]:
#import required libraries
import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Alignment
from openpyxl.styles import Font
from openpyxl.chart import BarChart, Reference
from xlsxwriter.utility import xl_rowcol_to_cell
import string
from datetime import datetime
import win32com.client
from IPython.display import display, HTML


#Reading Inventory status file starting from row 4
ef = pd.read_excel('K:\SnW Project\Inventory Report\Inventory status.xlsx',skiprows = range(3))  
ef = ef.rename(columns = {"Product Description":"Product"}) # renaming columns name to join the tables
ef[['Product ID','Product','Status','Status Description','Units']]
#Creating Pivot against each game for book status 
report_table = ef.pivot_table(index='Product',columns='Status Description',values='Units',aggfunc='sum').round(0)

#Reading Activation File starting from row 5
ef1 = pd.read_excel('K:\SnW Project\Inventory Report\Activations.xlsx',skiprows = range(4)) 
dict = {"Product":"Product ID","Unnamed: 2":"Product"} # renaming columns name to join the tables 
ef1.rename(columns = dict,inplace = True)
report_table1 = ef1.loc[ef1["Net Activations"] > 1000, ["Product ID","Product","Net Activations","Int. Commission"]]

#Reading SnW Status Report starting from row 10
report_table2 = pd.read_excel('K:\SnW Project\Inventory Report\Product Status.xlsx',skiprows = range(9),usecols = ['Product ID','Product Name','Start Prize Payment Date','Unnamed: 23','Total Prizes (Cash & Merch)','Total Cash Prizes Paid','Outstanding Cash Liability'])
dict = {"Product Name":"Product","Unnamed: 23":"PrintRun"} # renaming columns name to join the tables 
report_table2.rename(columns = dict,inplace = True)

Books = pd.read_excel('K:\SnW Project\Inventory Report\SnW Book.xlsx',usecols = ['Price Point','Price Point (Value)','Tickets in Book','Value of Books'])

#Concat all to reports into the final report  
all_data = pd.merge(report_table, report_table1,'left', on = ['Product'])
report = pd.merge(all_data, report_table2,'left', on = ['Product'])

#filtered the active games from table from activations
Final_report = report.loc[report["Int. Commission"] > 0, ["Product ID_y","Product","Net Activations","PrintRun","Total Prizes (Cash & Merch)","Total Cash Prizes Paid","Books arrive in the warehouse","Books have arrived at the Retailer and have been received on the terminal","Books have been distributed from warehouse and are on the way to the Retailer","Books have been sold to a retailer and are ready to be sold to players and validated",]]
dict = {"Product ID_y":"Product ID","Total Sales Amount":"Print Run","Total Prizes (Cash & Merch)":"Total Prizes","Total Cash Prizes Paid":"Total Prizes(Paid)","Books arrive in the warehouse":"Books(Stock at Warehouse)","Books have arrived at the Retailer and have been received on the terminal":"Books(at Retailer)","Books have been distributed from warehouse and are on the way to the Retailer":"Books in Transit","Books have been sold to a retailer and are ready to be sold to players and validated":"Books Activated"}
Final_report.rename(columns = dict,inplace = True) #renaming all required columns as per final output

Inv_report = report.loc[report["Net Activations"] > 1000,["Product ID_y","Product","Start Prize Payment Date","PrintRun","Net Activations","Books arrive in the warehouse","Books have arrived at the Retailer and have been received on the terminal","Books have been distributed from warehouse and are on the way to the Retailer","Books have been sold to a retailer and are ready to be sold to players and validated","Books are stolen after activation","Books are stolen from a Retailer, books from ready, available, transit status can move to stolen"]]
Inv_report['Total stock at Retailer'] = Inv_report['Books have been distributed from warehouse and are on the way to the Retailer']+Inv_report['Books have arrived at the Retailer and have been received on the terminal']

Inv_report['Price Point'] = Inv_report['Product'].str[:3]
Inv_report2 = pd.merge(Inv_report,Books,'left', on =['Price Point'])
Inv_report2['Activations(Books)'] = Inv_report2['Net Activations'] / Inv_report2['Value of Books']
Inv_report2['Stock in Network(Weeks)'] = round(Inv_report2['Total stock at Retailer'] / Inv_report2['Activations(Books)'],2)
Inv_report2['Total'] = Inv_report2['Books arrive in the warehouse'] + Inv_report2['Books have arrived at the Retailer and have been received on the terminal'] + Inv_report2['Books have been distributed from warehouse and are on the way to the Retailer']+Inv_report2["Books have been sold to a retailer and are ready to be sold to players and validated"]
Inv_report2['Sell Through(%)'] = round(Inv_report2['Books have been sold to a retailer and are ready to be sold to players and validated']/Inv_report2['Total'] * 100,2)
Inv_report2['Stock at warehouse(Week)'] = round(Inv_report2['Books arrive in the warehouse'] / Inv_report2['Activations(Books)'],2)
Inv_report2['Game Start Date'] = pd.to_datetime(Inv_report2['Start Prize Payment Date']).dt.date

        
dict = {"Product ID_y":"Product ID","Books arrive in the warehouse":"Books(at warehouse)","Books have been distributed from warehouse and are on the way to the Retailer":"In Transit","Books have been sold to a retailer and are ready to be sold to players and validated":"Activated","Books are stolen after activation":"Lost/Stolen (After Activation)","Books are stolen from a Retailer, books from ready, available, transit status can move to stolen":"Lost/Stolen(Before Activation)","Books have arrived at the Retailer and have been received on the terminal":"Books(Received by Retailer)"}
Inv_report2.rename(columns = dict,inplace = True)

Final_Inv = Inv_report2[["Product ID","Product","Net Activations","Books(at warehouse)","Books(Received by Retailer)","In Transit","Activated","Lost/Stolen (After Activation)","Lost/Stolen(Before Activation)","Total stock at Retailer","Activations(Books)","Stock in Network(Weeks)","Sell Through(%)","Stock at warehouse(Week)","Game Start Date"]]


with pd.ExcelWriter('K:\SnW Project\Inventory Report\Output Excels\Game Report(Friday)1.xlsx') as writer:
    Final_report.style.set_properties(**{'text-align': 'center'}).to_excel(writer,sheet_name ='Final_report',index = 0, startrow = 1)
    Final_Inv.style.set_properties(**{'text-align': 'center'}).to_excel(writer,sheet_name ='Inv_report',index = 0,startrow = 1)
    workbook = writer.book
    worksheet = writer.sheets['Final_report']

    worksheet1 = writer.sheets['Inv_report'] 
    
    worksheet.set_column('A:J',20)
    worksheet1.set_column('A:O',13)
    worksheet1.set_column('B:B',19)
#Creating header function 
    header_format = workbook.add_format({
        "valign":"vcenter",
        "align":"center",
        "bg_color":"#951F06",
        "bold": True,
        "text_wrap": True,
        "font_color":"#FFFFFF"
        })
    title = "Game Report on  "  +  datetime.now().strftime('%d %b %Y')
    title1 = "Inventory Report on  "  +  datetime.now().strftime('%d %b %Y')
    format = workbook.add_format()
    format.set_font_size(20)
    format.set_font_color("#333333")
    worksheet.merge_range('A1:J1',title,format)
    worksheet1.merge_range('A1:O1',title1,format)  
    worksheet.set_row(1,15)   
    worksheet1.set_row(1,15)
    for col_num, value in enumerate(Final_report.columns.values):
        worksheet.write(1, col_num, value, header_format)
    for col_num, value in enumerate(Final_Inv.columns.values):
        worksheet1.write(1, col_num, value, header_format)
    
#Converted excel into one page(print) to pdf
o = win32com.client.Dispatch("Excel.Application")
o.Visible = False

wb_path = r'K:\SnW Project\Inventory Report\Output Excels\Game Report(Friday)1.xlsx'
wb = o.Workbooks.Open(wb_path)

ws_index_list = ['Final_report']#calling worksheet

path_to_pdf = r'K:\SnW Project\Inventory Report\Output Excels\Game Report(Friday).pdf'#pdf output path

print_area = 'A1:J50'

for index in ws_index_list:
    ws = wb.Worksheets['Final_report']
    ws.PageSetup.Zoom = False
    ws.PageSetup.FitToPagesTall = 1
    ws.PageSetup.FitToPagesWide = 1
    ws.PageSetup.PrintArea = print_area
    
wb.WorkSheets(ws_index_list).Select()
wb.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf)

ws_index_list1 = ['Inv_report']#calling worksheet

path_to_pdf1 = r'K:\SnW Project\Inventory Report\Output Excels\Inventory Report.pdf'#pdf output path

print_area1 = 'A1:O50'

for index in ws_index_list1:
    ws = wb.Worksheets['Inv_report']
    ws.PageSetup.Zoom = False
    ws.PageSetup.FitToPagesTall = 1
    ws.PageSetup.FitToPagesWide = 1
    ws.PageSetup.PrintArea = print_area1
    
wb.WorkSheets(ws_index_list1).Select()
wb.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf1)

o.ActiveWorkbook.Save()
o.Quit() 
 


PermissionError: [Errno 13] Permission denied: 'K:\\SnW Project\\Inventory Report\\Output Excels\\Game Report(Friday)1.xlsx'